NODO #2 DE LA RED

In [1]:
class Block():
    def __init__(self, index, transactions, timestamp, previous_hash):
        """
        Constructor for the class Block
        index --> unique ID number for the block
        transaction --> LIST of transaction
        timestamp --> creation time of the Block
        previous_hash --> hash value of the previous Block

        """
        self.index = index
        self.transactions = transactions
        self.timestamp = timestamp
        self.previous_hash = previous_hash
        
    
    def compute_hash(self):
        """
        calculate hash --> hash sobre toda la información del bloque
        """
        # creamos un json del bloque 
        block_string = json.dumps(self.__dict__, sort_keys = True)
        
        # creamos hash del json creado
        hashed_block_string = hashlib.sha256(block_string.encode('utf-8')).hexdigest()
        
        return hashed_block_string

In [12]:
class Blockchain():
    def __init__(self, difficulty=2):
        """
        difficulty --> para que solo ponga dos ceros en el hash
        """
        self.unconfirmed_transactions = []
        self.chain = []
        self.create_genesis_block()
        self.difficulty = difficulty
        
    def create_genesis_block(self):
        genesis_block = Block(0, [], time.time(), "0")
        genesis_block.current_hash = genesis_block.compute_hash()
        self.chain.append(genesis_block)
        
    def add_genesis_block(self, genesis_block):
        # borramos el bloque de geneisis incicial y metemos el del otro nodo
        self.chain[0] = genesis_block
    
    def proof_of_work(self, bloque_nuevo):
        bloque_nuevo.nonce = 0 # añadimos atributo nonce
        while(True):
            hash_del_bloque = bloque_nuevo.compute_hash()
            first_caracters = hash_del_bloque[0 : (self.difficulty)]

            if first_caracters != "0"*self.difficulty:
                bloque_nuevo.nonce += 1
            else:
                break
        return hash_del_bloque
    
    def is_valid_proof(self, nuevo_hash, bloque):
        # hash del bloque = nuevo_hash
        
        if bloque.compute_hash() == nuevo_hash:
            hash_del_bloque = nuevo_hash
            first_caracters = hash_del_bloque[0 : (self.difficulty)]
            if first_caracters == "0"*self.difficulty:
                return True
            else:
                return False
        else:
            return False
    
    def append_block(self, nuevo_hash, bloque):
        if bloque.previous_hash == self.last_block.current_hash:
            if self.is_valid_proof(nuevo_hash, bloque):
                bloque.current_hash = nuevo_hash
                self.chain.append(bloque)
                return True
            else:
                return False
        else:
            return False
    
    def add_new_transaction(self, transaction):
        self.unconfirmed_transactions.append(transaction)
            
    @property
    def last_block(self):
        return self.chain[-1]
    
    # validar bloque con transaccion sin confirmar
    def mine(self):
        if len(self.unconfirmed_transactions) == 0:
            return False
        else:
            new_block = Block(index = self.last_block.index + 1, transactions = self.unconfirmed_transactions, timestamp = time.time(), previous_hash = self.last_block.current_hash)
            new_hash = self.proof_of_work(new_block)
            if self.append_block(new_hash, new_block):
                self.unconfirmed_transactions = []
                return new_block.index
            else:
                return False
    
    def check_chain(self, blockchain):
        cont = 0
        for block in blockchain:
            if cont != 0:
                block_anterior = blockchain[cont-1]
                current_hash_check = block.current_hash
                delattr(block, "current_hash")

                if self.is_valid_proof(current_hash_check, block):
                    if block.previous_hash == block_anterior.current_hash:
                        block.current_hash = current_hash_check
                        return True
                    else:
                        False
                else:
                    False
            cont = cont + 1

In [13]:
from flask import Flask, request
import json
import requests
import time
import hashlib

app = Flask(__name__)

blockchain = Blockchain()
peers = set()

@app.route("/")
def index():
    return "BIENVENIDO A LA API REST PARA LA PRÁCTICA DE BLOCKCHAIN"

@app.route("/new_transaction", methods = ['POST'])
def new_transaction():
    datos = request.get_json()
    
    autor = datos['author']
    contenido = datos['content']
    
    if autor == '' or contenido == '':
        return "Invalid data", 404
    else:
        timestamp = time.time()
        datos["time"] = timestamp
        blockchain.add_new_transaction(datos)
        return "Success", 201

@app.route("/chain", methods=['GET'])
def get_chain():
    global blockchain, peers
    
    diccionario = {}
    diccionario["length"] = len(blockchain.chain)
    diccionario["chain"] = [b.__dict__ for b in blockchain.chain]
    diccionario["peers"] = list(peers)
    
    return diccionario

@app.route("/mine", methods=['GET'])
def mine():
    global blockchain, peers
    index = 0
    
    # miramos si hace falta actualizar cadena
    if consensus():
        print("La cadena no era la más larga --> la hemos actualziado")
    else:
        print("nos quedeamos con la cadena actual")
    
    unconfirmed_transactions_backup = blockchain.unconfirmed_transactions
    
    out = blockchain.mine()
    if out == False:
        print("No hay transacciones para validar")
        return "No hay transacciones que validar"
    else:
        index = out
        print("Block mined: " + str(index))
        
        # hay una cadena MÁS larga --> ha minado a la vez y es mas grande
        if consensus() == True:
            print("Consensus despues de minado, no era la más larga")

            # restaurar la transacciones
            blockchain.unconfirmed_transactions = unconfirmed_transactions_backup

            return "El bloque ha sido descartado, hay que minar de nuevo"
        else:
            print("Consensus despues de minado, si era la más larga")
            announce_new_block(blockchain.chain[index], request)

            return "El bloque se ha minado exitosamente, indice del bloque: " + str(index)

@app.route("/pending_transactions", methods=['GET'])
def pending_transactions():
    return str(blockchain.unconfirmed_transactions)

@app.route("/register_new_node", methods = ['POST'])
def register_new_node():
    datos = request.get_json()
    
    node_ip = datos['new_node_address']
    
    if node_ip == '' :
        return "Invalid data", 404
    else:
        peers.add(node_ip)
        return get_chain()

@app.route("/add_peer", methods = ['POST'])
def update_peers_after_new_node_enters_blockchain():
    global peers
    datos = request.get_json()
    
    new_node = datos['new_node_address']
    print(new_node)
    print("hola")
    
    if new_node == '' :
        return "Invalid data", 404
    else:
        print("hola2")
        peers.add(new_node)
        
        for peer in peers:
            print(peer)
    
@app.route("/register_with_existing_node", methods = ['POST'])
def register_with_existing_node():
    # para poder cambiar el valor de la variable global
    global peers, blockchain
    
    datos = request.get_json()
    node_ip_existente_en_la_red = datos['node_address']
    
    if node_ip_existente_en_la_red == '' :
        return "Invalid data", 404
    
    data = {"new_node_address": request.host}
    headers = {'Content-Type': "application/json"}
    
    url = f"http://{node_ip_existente_en_la_red}/register_new_node"
    response = requests.post(url, data=json.dumps(data), headers=headers)
    
    if response.status_code == 200:
        chain_dump = response.json()['chain']
        peer_dump = response.json()['peers']
        
        # para poder hacer uso de las funciones de blockchain
        nueva_blockchain = Blockchain()
        cont = 0
        
        # cadena del nuevo nodo
        for bloque in chain_dump:
            indice = int(bloque["index"])
            transacciones = bloque["transactions"]
            timestamp = bloque["timestamp"]
            previous_hash = bloque["previous_hash"]
            current_hash = bloque["current_hash"]
            
            # necesitmaos crear un nuevo bloque para añadir a la cadena
            nuevo_bloque = Block(indice, transacciones, timestamp, previous_hash)
            
            # mirar si es bloque génensis o no --> por el nonce
            if cont != 0:
                nonce = int(bloque["nonce"])
                nuevo_bloque.nonce = nonce

                if nueva_blockchain.append_block(current_hash, nuevo_bloque) == False:
                    return "Invalid chain", 404
            else:
                nuevo_bloque.current_hash = current_hash
                nueva_blockchain.add_genesis_block(nuevo_bloque)
            
            cont = cont + 1
            
        # hay que añadir TODOS menos la propia IP
        for peer in peer_dump:
            if peer != request.host:
                peers.add(peer)
        
        # añadir el nodo que estaba en la red
        peers.add(node_ip_existente_en_la_red)
        
        for peer in peers:
            if peer != request.host:
                if peer != node_ip_existente_en_la_red:
                    data_peer = {"new_node_address": request.host}
                    headers_peer = {'Content-Type': "application/json"}

                    url = f"http://{peer}/add_peer"
                    response = requests.post(url, data=json.dumps(data_peer), headers=headers_peer)
        
        blockchain = nueva_blockchain
            
        return "Registration successful", 200
    else:
        return response.content, response.status_code
        
@app.route("/add_block", methods = ['POST'])
def add_block_from_other_nodes_to_add_to_current_blockchain():
    global blockchain, peers
    
    datos = request.get_json()
    
    index = datos['index']
    transactions = datos['transactions']
    timestamp = datos['timestamp']
    previous_hash = datos['previous_hash']
    current_hash = datos['current_hash']
    
    nuevo_bloque = Block(index, transactions, timestamp, previous_hash)
    nuevo_bloque.nonce = datos['nonce']
    
    if blockchain.append_block(current_hash, nuevo_bloque):
        print("APPENDED")
        return "Block appended to the chain", 201
    else:
        print("NOT APPENDED")
        return "Block discarded", 400
    
def announce_new_block(new_mined_block, request: Flask.request_class):
    for peer in peers:
        if peer != request.host:
            data = json.dumps(new_mined_block.__dict__,sort_keys=True)
            headers = {'Content-Type': "application/json"}
            url = f"http://{peer}/add_block"
            
            response = requests.request("POST", url, data=data, headers=headers)

def consensus():
    global blockchain, peers
    
    longest_chain = None
    current_chain_len = len(blockchain.chain)
        
    for peer in peers:
        url = f"http://{peer}/chain"
        print(url)
        data = {}
        header = {}
        response = requests.request("GET", url, headers=header, data=data)

        if response.status_code == 200:
            response_chain_length = response.json()["length"]
            response_chain = response.json()["chain"]

        # la cadena enconctrada tiene una mayor longitud
        if response_chain_length > current_chain_len:
            longest_chain = response_chain
            current_chain_len = response_chain_length
    
    
    # se ha llegado a conseenso -> hay que actualizar la cadena
    # porque no es la más larga
    if longest_chain:
        blockchain = longest_chain
        return True
        
    return False
    
if __name__ == '__main__':
    app.run(port=8086)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8086/ (Press CTRL+C to quit)
127.0.0.1 - - [17/May/2022 11:22:16] "POST /register_with_existing_node HTTP/1.1" 200 -
127.0.0.1 - - [17/May/2022 11:22:39] "GET /chain HTTP/1.1" 200 -
127.0.0.1 - - [17/May/2022 11:22:46] "POST /register_new_node HTTP/1.1" 200 -
127.0.0.1 - - [17/May/2022 11:22:53] "GET /chain HTTP/1.1" 200 -
127.0.0.1 - - [17/May/2022 11:23:11] "POST /new_transaction HTTP/1.1" 201 -
127.0.0.1 - - [17/May/2022 11:23:13] "GET /mine HTTP/1.1" 200 -


http://127.0.0.1:8087/chain
http://127.0.0.1:8085/chain
nos quedeamos con la cadena actual
Block mined: 3
http://127.0.0.1:8087/chain
http://127.0.0.1:8085/chain
Consensus despues de minado, si era la más larga


127.0.0.1 - - [17/May/2022 11:23:17] "GET /chain HTTP/1.1" 200 -


In [ ]:
@app.route("/register_with_existing_node",methods = ["POST"])
def register_existing_node():
    global peers, blockchain
    node_address = request.get_json()["node_address"]
    # Constructs a POST request to register_new_node endpoint
    url = f"http://{node_address}/register_new_node"
    payload = json.dumps({
    "new_node_address": request.host
    })
    headers = {
    'Content-Type': 'application/json'
    }
    # Sends the request and returns the answer
    response = requests.request("POST", url, headers=headers, data=payload)
    # Depending on the status code of response
    if response.status_code == 201:
        # Updates chain and peers
        chain_dumps = response.json()["chain"]
        peers_dumps = response.json()["peers"]
        # Setting the peers
        for block_dumps in chain_dumps:
            # If it the first block (genesis) then it is added automately
            block = Block(int(block_dumps["index"]),block_dumps["transactions"],float(block_dumps["timestamp"]),block_dumps["previous_hash"])
            block.nonce = int(block_dumps["nonce"])
            
            # Depending on whether it is the first block or not
            if block_dumps == chain_dumps[0]:
                # Creates a blockchain with the genesis block recieved
                block.current_hash = block_dumps["current_hash"]
                blockchain_dumps = Blockchain(genesis_block=block)
            else:
                # Attemps to add the block tothe blockchain
                was_added = blockchain_dumps.append_block(block,block_dumps["current_hash"])
                if not was_added:
                    return (f"Validation of chain failed ({block} - {block}", 400) # The validation failed
        
        # The completition of new blockchain was succesful
        blockchain = blockchain_dumps
        # Sets the difficulty of the blockchain depending on the number of zeros of last block
        difficulty = 0
        char = block.current_hash[0]
        while char == '0':
            difficulty += 1
            char = block.current_hash[difficulty]   
        blockchain.difficulty = difficulty # Asigns the difficulty to the blockchain
        # Creates a new set of peers
        peers = peers_dumps
        return ("Registration successful", 200)
    else:
        # The post request failed
        return response.content, response.status_code

In [ ]:
def consensus() -> bool :
    """ 
        Function that implements the consensus logic to replicate the blockchain in a node.
        The function looks for the longest chain in the network, and then the actual blockchain of the node is replace by this one found.
        Returns true if a consensus is reached, False otherwise. 
    """
    global blockchain
    longest_chain = None
    current_len = len(blockchain.chain)
    for node in peers:
        try:
            # A GET request to endpoint /chain is sent to the node direction
            url = f"http://{node}/chain"
            payload={}
            headers = {}
            response = requests.request("GET", url, headers=headers, data=payload)
            if response.status_code == 200:
                length = response.json()["length"]
                chain = response.json()["chain"]
                if length > current_len:
                    # The chain is longer than the current one
                    longest_chain = chain
                    current_len = length
        except ConnectionRefusedError:
            pass
    if longest_chain is not None:
        # The blockchain of node is now the longest found
        blockchain = longest_chain
        return True
    else:
        return False

@app.route("/add_block",methods = ["POST"])
def add_block():
    try:
        # Attempts to get the required fields
        index = request.get_json()["index"]
        transactions = request.get_json()["transactions"]
        timestamp = request.get_json()["timestamp"]
        previous_hash = request.get_json()["previous_hash"]
        current_hash = request.get_json()["current_hash"]
    except KeyError: 
        return ("Invalid data", 400)
    else:
        # Constructs a block
        block = Block(index,transactions,timestamp,previous_hash)
        # Tries to append the block to the blockchain
        global blockchain
        added = blockchain.append_block(block,current_hash)
        if added:
            return ("Success", 200)
        else: 
            return ("The block could not be appended to the blockchain.",400)

def announce_new_block(block: Block, request: Flask.request_class):
    """ Announces a new block to the rest of nodes of the Blockchain network. Arguments:
        * block: Block object
        * request: Request object of Flask 
        The rest of the nodes only have to verify the PoW and add the string. 
    """
    # Iterates for each node in peers variable
    for node in peers:
        if node != request.host_url:
            try:
                # Sends a POST request to /add_block
                url = f"http://{node}/add_block"
                payload = json.dumps(block._dict_,sort_keys=True)
                headers = {
                'Content-Type': 'application/json'
                }
                requests.request("POST", url, headers=headers, data=payload)
            except ConnectionRefusedError:
                # In case the node is not responding
                pass